# Genie Space Migration Tool

This notebook allows you to export a Genie space from one Databricks workspace and deploy it to another workspace.

## Features
- Cloud agnostic (Azure, AWS, GCP Databricks)
- Supports both PAT and Service Principal authentication
- Exports complete Genie space configuration (code, settings, instructions, joins)
- Deploys to target workspace with full configuration

## Prerequisites
- Admin access to both source and target workspaces
- Valid authentication credentials (PAT or Service Principal)
- Important: If the script would be run on Databricks, then use Secret Scopes for passing credentials.

## Configuration


In [ ]:
import requests
import json
import os

# ==================== OPTION 1: Load from Config File ====================
# Set CONFIG_FILE to your JSON config file path, or None to use manual configuration below
CONFIG_FILE = "config.json"  # Set to None to use manual configuration

# ==================== OPTION 2: Manual Configuration ====================
# These defaults will be used if CONFIG_FILE is None or file not found

# Source Workspace Configuration
SOURCE_WORKSPACE_URL = ""  # Example: "https://adb-1234567890.7.azuredatabricks.net"
SOURCE_GENIE_SPACE_ID = ""  # Example: "01ef1234567890abcdef1234567890ab"
SOURCE_AUTH_TYPE = "PAT"  # Options: "PAT" or "SERVICE_PRINCIPAL"
SOURCE_PAT = ""  # Example: "dapi1234567890abcdef1234567890ab"
SOURCE_SP_CLIENT_ID = ""  # Example: "12345678-1234-1234-1234-123456789012"
SOURCE_SP_CLIENT_SECRET = ""

# Target Workspace Configuration
TARGET_WORKSPACE_URL = ""  # Example: "https://dbc-abcdef12-3456.cloud.databricks.com"
TARGET_AUTH_TYPE = "PAT"  # Options: "PAT" or "SERVICE_PRINCIPAL"
TARGET_PAT = ""  # Example: "dapi1234567890abcdef1234567890ab"
TARGET_SP_CLIENT_ID = ""  # Example: "12345678-1234-1234-1234-123456789012"
TARGET_SP_CLIENT_SECRET = ""

# REQUIRED: Target SQL Warehouse ID (different from source workspace)
TARGET_SQL_WAREHOUSE_ID = ""  # Example: "abc123def456"

# Optional: Override title and description
TARGET_TITLE_OVERRIDE = None  # Set to override, or None to keep original
TARGET_DESCRIPTION_OVERRIDE = None  # Set to override, or None to keep original

# Action: "CREATE" or "UPDATE"
ACTION = "CREATE"
EXISTING_TARGET_SPACE_ID = ""  # Required if ACTION is "UPDATE"

# ==================== Load Config File (if specified) ====================
if CONFIG_FILE and os.path.exists(CONFIG_FILE):
    print(f"Loading configuration from: {CONFIG_FILE}")
    with open(CONFIG_FILE, 'r') as f:
        config = json.load(f)
    
    # Override with config file values
    SOURCE_WORKSPACE_URL = config.get('source_workspace_url', SOURCE_WORKSPACE_URL)
    SOURCE_GENIE_SPACE_ID = config.get('source_genie_space_id', SOURCE_GENIE_SPACE_ID)
    SOURCE_AUTH_TYPE = config.get('source_auth_type', SOURCE_AUTH_TYPE)
    SOURCE_PAT = config.get('source_pat', SOURCE_PAT)
    SOURCE_SP_CLIENT_ID = config.get('source_sp_client_id', SOURCE_SP_CLIENT_ID)
    SOURCE_SP_CLIENT_SECRET = config.get('source_sp_client_secret', SOURCE_SP_CLIENT_SECRET)
    
    TARGET_WORKSPACE_URL = config.get('target_workspace_url', TARGET_WORKSPACE_URL)
    TARGET_AUTH_TYPE = config.get('target_auth_type', TARGET_AUTH_TYPE)
    TARGET_PAT = config.get('target_pat', TARGET_PAT)
    TARGET_SP_CLIENT_ID = config.get('target_sp_client_id', TARGET_SP_CLIENT_ID)
    TARGET_SP_CLIENT_SECRET = config.get('target_sp_client_secret', TARGET_SP_CLIENT_SECRET)
    
    TARGET_SQL_WAREHOUSE_ID = config.get('target_sql_warehouse_id', TARGET_SQL_WAREHOUSE_ID)
    TARGET_TITLE_OVERRIDE = config.get('target_space_display_name') or TARGET_TITLE_OVERRIDE
    TARGET_DESCRIPTION_OVERRIDE = config.get('target_space_description') or TARGET_DESCRIPTION_OVERRIDE
    
    ACTION = config.get('action', ACTION)
    EXISTING_TARGET_SPACE_ID = config.get('existing_target_space_id', EXISTING_TARGET_SPACE_ID)
    
    print(f"Configuration loaded successfully from {CONFIG_FILE}")
elif CONFIG_FILE:
    print(f"Warning: Config file '{CONFIG_FILE}' not found. Using manual configuration.")
    print("Configuration loaded from manual settings")
else:
    print("Configuration loaded from manual settings")

# Display current configuration
print(f"\nCurrent Configuration:")
print(f"  Source: {SOURCE_WORKSPACE_URL}")
print(f"  Target: {TARGET_WORKSPACE_URL}")
print(f"  Action: {ACTION}")


## Step 1: Get Source Genie Space


In [ ]:
print("=" * 80)
print("STEP 1: FETCHING GENIE SPACE FROM SOURCE WORKSPACE")
print("=" * 80)
print()

print(f"Authenticating to source workspace using {SOURCE_AUTH_TYPE}...")

# Build source authentication headers
source_headers = {"Content-Type": "application/json"}

if SOURCE_AUTH_TYPE == "PAT":
    source_headers["Authorization"] = f"Bearer {SOURCE_PAT}"
elif SOURCE_AUTH_TYPE == "SERVICE_PRINCIPAL":
    # Get OAuth token for Service Principal
    source_workspace_url = SOURCE_WORKSPACE_URL.rstrip('/')
    token_url = f"{source_workspace_url}/oidc/v1/token"
    
    token_data = {
        'grant_type': 'client_credentials',
        'scope': 'all-apis'
    }
    
    token_response = requests.post(
        token_url,
        auth=(SOURCE_SP_CLIENT_ID, SOURCE_SP_CLIENT_SECRET),
        data=token_data,
        headers={'Content-Type': 'application/x-www-form-urlencoded'}
    )
    token_response.raise_for_status()
    source_token = token_response.json()['access_token']
    source_headers["Authorization"] = f"Bearer {source_token}"
else:
    raise ValueError(f"Invalid SOURCE_AUTH_TYPE: {SOURCE_AUTH_TYPE}")

print("Authentication successful")

# Get Genie space with serialized configuration
print(f"\nFetching Genie space: {SOURCE_GENIE_SPACE_ID}...")

source_workspace_url = SOURCE_WORKSPACE_URL.rstrip('/')
get_url = f"{source_workspace_url}/api/2.0/genie/spaces/{SOURCE_GENIE_SPACE_ID}?include_serialized_space=true"

get_response = requests.get(get_url, headers=source_headers)
get_response.raise_for_status()
source_space = get_response.json()

print("Genie space retrieved successfully")

# Display space summary
print("\n" + "=" * 80)
print("SOURCE GENIE SPACE SUMMARY")
print("=" * 80)
print(f"Title: {source_space.get('title', '')}")
print(f"Description: {source_space.get('description', '')}")
print(f"Warehouse ID: {source_space.get('warehouse_id', '')}")

print("\nDEBUG: Full API response:")
print(json.dumps(source_space, indent=2))


## Step 2: Prepare Target Configuration


In [ ]:
print("=" * 80)
print("STEP 2: PREPARING CONFIGURATION FOR TARGET WORKSPACE")
print("=" * 80)
print()

# Use values from source or apply overrides
target_title = TARGET_TITLE_OVERRIDE if TARGET_TITLE_OVERRIDE else source_space.get('title', '')
target_description = TARGET_DESCRIPTION_OVERRIDE if TARGET_DESCRIPTION_OVERRIDE else source_space.get('description', '')

print(f"Target Title: {target_title}")
print(f"Target Description: {target_description}")
print(f"Target SQL Warehouse ID: {TARGET_SQL_WAREHOUSE_ID}")


## Step 3: Deploy to Target Workspace


In [ ]:
print("=" * 80)
print("STEP 3: DEPLOYING GENIE SPACE TO TARGET WORKSPACE")
print("=" * 80)
print()

print(f"Authenticating to target workspace using {TARGET_AUTH_TYPE}...")

# Build target authentication headers
target_headers = {"Content-Type": "application/json"}

if TARGET_AUTH_TYPE == "PAT":
    target_headers["Authorization"] = f"Bearer {TARGET_PAT}"
elif TARGET_AUTH_TYPE == "SERVICE_PRINCIPAL":
    # Get OAuth token for Service Principal
    target_workspace_url = TARGET_WORKSPACE_URL.rstrip('/')
    token_url = f"{target_workspace_url}/oidc/v1/token"
    
    token_data = {
        'grant_type': 'client_credentials',
        'scope': 'all-apis'
    }
    
    token_response = requests.post(
        token_url,
        auth=(TARGET_SP_CLIENT_ID, TARGET_SP_CLIENT_SECRET),
        data=token_data,
        headers={'Content-Type': 'application/x-www-form-urlencoded'}
    )
    token_response.raise_for_status()
    target_token = token_response.json()['access_token']
    target_headers["Authorization"] = f"Bearer {target_token}"
else:
    raise ValueError(f"Invalid TARGET_AUTH_TYPE: {TARGET_AUTH_TYPE}")

print("Authentication successful")

# Prepare payload using API response as-is
target_workspace_url = TARGET_WORKSPACE_URL.rstrip('/')

if ACTION == "CREATE":
    print("\nCreating new Genie space in target workspace...")
    
    # Build request body
    request_body = {
        "warehouse_id": TARGET_SQL_WAREHOUSE_ID,
        "title": target_title,
        "description": target_description,
        "serialized_space": source_space.get('serialized_space', '')
    }
    
    print("\nDEBUG: Request payload:")
    print(json.dumps(request_body, indent=2))
    
    create_url = f"{target_workspace_url}/api/2.0/genie/spaces"
    create_response = requests.post(create_url, headers=target_headers, json=request_body)
    create_response.raise_for_status()
    result_space = create_response.json()
    
    print("Genie space created successfully")
    
elif ACTION == "UPDATE":
    if not EXISTING_TARGET_SPACE_ID:
        raise ValueError("EXISTING_TARGET_SPACE_ID is required when ACTION is 'UPDATE'")
    
    print(f"\nUpdating existing Genie space: {EXISTING_TARGET_SPACE_ID}...")
    
    # Build request body
    request_body = {
        "warehouse_id": TARGET_SQL_WAREHOUSE_ID,
        "title": target_title,
        "description": target_description,
        "serialized_space": source_space.get('serialized_space', '')
    }
    
    print("\nDEBUG: Request payload:")
    print(json.dumps(request_body, indent=2))
    
    update_url = f"{target_workspace_url}/api/2.0/genie/spaces/{EXISTING_TARGET_SPACE_ID}"
    update_response = requests.patch(update_url, headers=target_headers, json=request_body)
    update_response.raise_for_status()
    result_space = update_response.json()
    
    print("Genie space updated successfully")
    
else:
    raise ValueError(f"Invalid ACTION: {ACTION}. Must be 'CREATE' or 'UPDATE'")

# Display completion
print("\n" + "=" * 80)
print("MIGRATION COMPLETED SUCCESSFULLY")
print("=" * 80)
print(f"Source Space ID: {SOURCE_GENIE_SPACE_ID}")
print(f"Target Space ID: {result_space.get('space_id', '')}")
print(f"Target Workspace URL: {TARGET_WORKSPACE_URL}")
print("\nNew Genie Space URL:")
print(f"{TARGET_WORKSPACE_URL}/genie/spaces/{result_space.get('space_id', '')}")
print()
